In [100]:
#Беру только по 2500 предложений, потому что, если брать больше, происходит Memory Error. Видимо, мой комп совсем плох.

In [101]:
import math
from math import log

In [102]:
import csv
with open('quora_question_pairs_rus.csv', 'r', encoding = 'utf-8') as r:
    reader = csv.reader(r)
    file = list(reader)

In [103]:
import pymorphy2 as pm2
pmm = pm2.MorphAnalyzer()
import re

In [104]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [105]:
docs = []
for indx, row in enumerate(file):
    if indx > 0:
        docs.append([row[2], row[3]])

In [7]:
#corpus = []
#for indx, row in enumerate(file):
#    if indx > 0:
#        doc = row[2]
#        doc = re.sub(r'[^\w\s]', '', doc)
#        doc = re.sub(r'\d', '', doc)
#        doc = re.sub(r'[A-Za-z]', '', doc)
#        doc_arr = [pmm.normal_forms(x)[0] for x in doc.split() if x not in russian_stopwords]
#        doc = ' '.join(doc_arr)
#        corpus.append(doc)

In [8]:
#queries = []
#for indx, row in enumerate(file):
#    if indx > 0:
#        query = row[1]
#        query = re.sub(r'[^\w\s]', '', query)
#        query = re.sub(r'\d', '', query)
#        query = re.sub(r'[A-Za-z]', '', query)
#        query_arr = [pmm.normal_forms(x)[0] for x in query.split() if x not in russian_stopwords]
#        query = ' '.join(query_arr)
#        queries.append(query)

In [9]:
#with open('corpus.txt', 'r', encoding = 'utf-8') as r:
#    corpus = r.readlines()

In [10]:
#with open('queries.txt', 'r', encoding = 'utf-8') as r:
#    queries = r.readlines()

In [11]:
#corp = corpus[:2500]

In [12]:
#quer = queries[:2500]

In [13]:
#with open('corp.txt', 'w', encoding = 'utf-8') as w:
#    for i in corp:
#        w.write(i)

In [14]:
#with open('quer.txt', 'w', encoding = 'utf-8') as w:
#    for i in quer:
#        w.write(i)

In [106]:
with open('corp.txt', 'r', encoding = 'utf-8') as r:
    corp = r.readlines()

In [107]:
with open('quer.txt', 'r', encoding = 'utf-8') as r:
    quer = r.readlines()

## Подготовительная работа. Подсчет переменных для формулы

In [108]:
#кол-во документов в коллекции
N = len(corp)
N

2500

In [109]:
#длины документов
lens_d = []
for doc in corp:
    doc = doc.split()
    lens_d.append(len(doc))
lens_d[:5]

[7, 5, 5, 4, 8]

In [110]:
#средняя длина документа
avgdl = sum(lens_d)/len(lens_d)
avgdl

5.9084

In [111]:
#константы
k = 2
b = 0.75

In [112]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
import numpy as np

In [113]:
X = vectorizer.fit_transform(corp)
#уникальные слова в корпусе
words = vectorizer.get_feature_names()

In [114]:
matrix = X.toarray()
#обратная матрица
rev_matrix = np.transpose(matrix)

In [115]:
#кол-во документов, где есть слово
number_of_docs = []
for word in rev_matrix:
    count = 0
    for i in word:
        if i != 0:
            count += 1
    number_of_docs.append(count)

In [116]:
TFs = []
for word in words:
    arr = []
    for sent in corp:
        TF = 0
        sent = sent.split()
        if word in sent:
            num = 0
            for s in sent:
                if word == s:
                    num += 1
            TF = num / len(sent)
        arr.append(TF)
    TFs.append(arr)

In [117]:
len(TFs)

4304

In [118]:
#Создание матрицы с BM25
bm_mtrx = []
for index, word in enumerate(words):
    word_bms = []
    for indx, doc in enumerate(corp):
        l = float(lens_d[indx])
        n = float(number_of_docs[index])
        TF = float(TFs[index][indx])
        bm = math.log((N - n + 0.5) / (n + 0.5)) * (TF * (k + 1)) / (TF + (k * (1 - b + b * l / avgdl)))
        word_bms.append(bm)
    bm_mtrx.append(word_bms)

In [119]:
len(bm_mtrx)

4304

# Задача 1

## Способ №1. Подсчет метрики по формуле для каждой пары слово-документ

In [120]:
%%time

success = []
q_number = 0

for q in quer:
    bms = []
    
    for i in q.split():
        for indx, word in enumerate(words):
            if i == word:
                bm = bm_mtrx[indx]
                bms.append(bm)
                
    if len(bms) > 0:
        q_bm = sum(np.array(bms))
        for_range = list(zip(q_bm, docs))
        sort_range = sorted(for_range, reverse=True)
        
        for indx, i in enumerate(sort_range):
            if indx < 5:
                if i[1][1] == '1':
                    success.append(1)
                    break
                    
    q_number += 1
        
accuracy = len(success) / q_number
print('Точность поиска = ', accuracy)

Точность поиска =  0.8968
Wall time: 34 s


## Способ №2. Подсчет метрики через умножение матрицы на вектор

In [121]:
import numpy as np

In [122]:
%%time
q_number = 0
success = []

for q in quer:
    vector = []
    
#создаем вектор запроса из нолей и единиц
    for word in words:
        if word in q.split():
            vector.append(1)
        else:
            vector.append(0)
        
    np_vector = np.asarray(vector)
    #умножаем матрицу bm25 на полученный вектор
    q_bm = np_vector.dot(np.asarray(bm_mtrx))

    for_range = list(zip(q_bm, docs))
    sort_range = sorted(for_range, reverse=True)
        
    for indx, i in enumerate(sort_range):
        if indx < 5:
            if i[1][1] == '1':
                success.append(1)
                break
                    
    q_number += 1
        
accuracy = len(success) / q_number
print('Точность поиска = ', accuracy)

Точность поиска =  0.902
Wall time: 26min 49s


# Задача 2

In [123]:
#ДИСКЛЕЙМЕР!!! Т.к. мой корпус состоит всего из 2500 предложений, в нем нет слов из запроса "рождественские каникулы". Поэтому я взяла другое словосочетание.

In [124]:
q_input = 'день рождения'
q_arr = [pmm.normal_forms(x)[0] for x in q_input.split() if x not in russian_stopwords]
bms_q = []
    
for i in q_input.split():
    for indx, word in enumerate(words):
        
        if i == word:
            bm = bm_mtrx[indx]
            bms_q.append(bm)

In [125]:
v = sum(np.array(bms_q))
v

array([0., 0., 0., ..., 0., 0., 0.])

In [126]:
for_range = list(zip(v, docs))

In [127]:
sort_range = sorted(for_range, reverse=True)

In [128]:
for indx, row in enumerate(sort_range):
    if indx < 10:
        print(row[1][0], ': ', row[0])

если бы у вас была возможность быть кем-то еще на один день, кто бы вы были :  2.7763629925786444
будет есть 2 000 калорий в день, чтобы я похудел :  2.7763629925786444
можно сжечь 1000 калорий каждый день, если да, как :  1.8811307833920714
что произойдет, если диктатура сохранится и в наши дни :  1.349112540919825
у меня есть мания, если я слушаю музыку в течение всего дня :  1.349112540919825
сколько раз нужно заниматься сексом в день :  1.349112540919825
сколько раз в день перекрывают руки часы :  1.349112540919825
сколько минут йоги можно сделать за один день :  1.349112540919825
сколько времени занимает квора в день :  1.349112540919825
здорово есть целый авокадо каждый день :  1.349112540919825


In [129]:
if_success = []
for indx, row in enumerate(sort_range):
    
    if indx < 5:
        if row[1][1] == '1':
            if_success.append('wow')
            
if 'wow' in if_success:
    print('Алгоритм точен!')
else:
    print('Алгоритм не точен.')

Алгоритм точен!


# Задача 3

## BM25, b=0.75

In [130]:
b = 0.75

In [131]:
#Создание матрицы с BM25
bm_mtrx = []
for index, word in enumerate(words):
    word_bms = []
    for indx, doc in enumerate(corp):
        l = float(lens_d[indx])
        n = float(number_of_docs[index])
        TF = float(TFs[index][indx])
        bm = math.log((N - n + 0.5) / (n + 0.5)) * (TF * (k + 1)) / (TF + (k * (1 - b + b * l / avgdl)))
        word_bms.append(bm)
    bm_mtrx.append(word_bms)

In [132]:
success = []
q_number = 0

for q in quer:
    bms = []
    
    for i in q.split():
        for indx, word in enumerate(words):
            if i == word:
                bm = bm_mtrx[indx]
                bms.append(bm)
                
    if len(bms) > 0:
        q_bm = sum(np.array(bms))
        for_range = list(zip(q_bm, docs))
        sort_range = sorted(for_range, reverse=True)
        
        for indx, i in enumerate(sort_range):
            if indx < 5:
                if i[1][1] == '1':
                    success.append(1)
                    break
                    
    q_number += 1
        
accuracy = len(success) / q_number
print('Точность поиска = ', accuracy)

Точность поиска =  0.8968


## BM15, b=0

In [133]:
b = 0

In [134]:
#Создание матрицы с BM25
bm_mtrx = []
for index, word in enumerate(words):
    word_bms = []
    for indx, doc in enumerate(corp):
        l = float(lens_d[indx])
        n = float(number_of_docs[index])
        TF = float(TFs[index][indx])
        bm = math.log((N - n + 0.5) / (n + 0.5)) * (TF * (k + 1)) / (TF + (k * (1 - b + b * l / avgdl)))
        word_bms.append(bm)
    bm_mtrx.append(word_bms)

In [135]:
success = []
q_number = 0

for q in quer:
    bms = []
    
    for i in q.split():
        for indx, word in enumerate(words):
            if i == word:
                bm = bm_mtrx[indx]
                bms.append(bm)
                
    if len(bms) > 0:
        q_bm = sum(np.array(bms))
        for_range = list(zip(q_bm, docs))
        sort_range = sorted(for_range, reverse=True)
        
        for indx, i in enumerate(sort_range):
            if indx < 5:
                if i[1][1] == '1':
                    success.append(1)
                    break
                    
    q_number += 1
        
accuracy = len(success) / q_number
print('Точность поиска = ', accuracy)

Точность поиска =  0.9032


## BM11, b=1

In [139]:
b = 1

In [140]:
#Создание матрицы с BM25
bm_mtrx = []
for index, word in enumerate(words):
    word_bms = []
    for indx, doc in enumerate(corp):
        l = float(lens_d[indx])
        n = float(number_of_docs[index])
        TF = float(TFs[index][indx])
        bm = math.log((N - n + 0.5) / (n + 0.5)) * ((TF * (k + 1)) / (TF + k * (1 - b + b * l / avgdl)))
        word_bms.append(bm)
    bm_mtrx.append(word_bms)

ZeroDivisionError: float division by zero

In [141]:
#BM11 не сработало из-за ошибки с делением на 0.